# Processing

## Filter images

This pice can be use to filter images from the metadata in Cohen's data set

Do not use this part if using the already selected images in the output dir "..._select_proc"; just skip to feature extraction

In [ ]:
import filter_images_from_CXR

In [ ]:
metadata_pth = "./repos/covid-chestxray-dataset/metadata.csv" # Meta info
imageDir = "./repos/covid-chestxray-dataset/images" # Directory of images
outputDir = './output/filtered_images' # Output directory to store selected images

#"went_icu" : "Y" 
#"in_icu" : "Y" 
#"finding": "COVID-19"
#"modality" : "X-ray"
#"survival": "N"
#"went_icu": ["N", 'match'],
#"match" to find the codition, "exclude" to avoid
conditions = {"finding": ["COVID-19", 'match'],
              "modality" : ["X-ray", 'match'],
              "went_icu": ["N", 'match'],
              "in_icu" : ["N", 'match'] }


save_imgs = True
del_files = True


filter_images_from_CXR.filter_(conditions, metadata_pth, imageDir, outputDir, save_imgs, del_files)

## Segment lungs (Cohen)

This piece will call the segmentation network (weights on the root folder) and output the segmented images

Do not use this part if using the already selected images in the output dir "..._select_proc"; just skip to feature extraction

In [ ]:
import segment_filtered_imgs

In [ ]:
model_path = './res_unet_seg.h5'
images_path = './output/filtered_images'
output_masks_path = './output/masks'
output_segmentation_path = "./output/not_went_icu"


CLAHE_load = False
CLAHE_clip_load = 3
CLAHE_grid_size_load = 4
CLAHE_save = True
CLAHE_clip_save = 3
CLAHE_grid_size_save = 64


segment_filtered_imgs.segment_filtered_images(model_path, images_path, output_masks_path, output_segmentation_path, 
                                                CLAHE_load, CLAHE_clip_load, CLAHE_grid_size_load, 
                                                CLAHE_save, CLAHE_clip_save, CLAHE_grid_size_save)

## Segment on BIMCV

need to invert colors of inverted X-Rays

In [ ]:
import segBIMCV

In [ ]:
model_path = './res_unet_seg.h5'
images_path = './datasets/BIMCV_processed/imgs'
output_masks_path = './output/masks'
output_segmentation_path = "./output/BIMCV/masked"


CLAHE_load = False
CLAHE_clip_load = 3
CLAHE_grid_size_load = 4
CLAHE_save = True
CLAHE_clip_save = 3
CLAHE_grid_size_save = 64


segBIMCV.segment_filtered_images(model_path, images_path, output_masks_path, output_segmentation_path, 
                                                CLAHE_load, CLAHE_clip_load, CLAHE_grid_size_load, 
                                                CLAHE_save, CLAHE_clip_save, CLAHE_grid_size_save)

# Augment images

Copy selected and processed images to a separate directory and apply augmentations to it

In [12]:
import imageio
import imgaug as ia
import os
from imgaug import augmenters as iaa
from PIL import Image


img_dir = './output/hanno_icu_selected_preproc_aug'

img_lst = os.listdir(img_dir)

augs = {
    'rotate': iaa.Affine(rotate=(-25, 25)),
    'pieceaff': iaa.PiecewiseAffine(scale=(0.01, 0.05)),
    'translate': iaa.Affine(translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)}),
    'shear': iaa.Affine(shear=(-10, 10))
}

reps = 1

for img in img_lst:
    for aug in augs:
        
        for rep in range(reps):
        
            image = imageio.imread(os.path.join(img_dir, img))

            image_aug = augs[aug](image=image)

            im = Image.fromarray(image_aug)

            im.save(os.path.join(img_dir,img[:-4] + '_' + aug + '_' + str(rep) + '.jpg'))
        
        
        
        

# Extract from last layer

In [13]:
import extract_from_last_layer 

## Extract features

In [17]:
features_output_path = './output/hanno_non_icu_selected_preproc_last.csv'
imgs_path = "./output/hanno_non_icu_selected_preproc/"
img_id_path = './output/hanno_non_icu_selected_preproc_last.txt'

extract_from_last_layer.extract_features(imgs_path,features_output_path,img_id_path)

images processed = 54 / 54
shape of features: (54, 18)
saved to: ../../.././output/hanno_non_icu_selected_preproc_last.csv


In [ ]:
#print features names to file

output_features_names_pth = './output/features_names.txt'

feature_names = {'Atelectasis': 0,
                   'Cardiomegaly': 1,
                   'Consolidation': 2,
                   'Edema': 3,
                   'Effusion': 4,
                   'Emphysema': 5,
                   'Enlarged Cardiomediastinum': 6,
                   'Fibrosis': 7,
                   'Fracture': 8,
                   'Hernia': 9,
                   'Infiltration': 10,
                   'Lung Lesion': 11,
                   'Lung Opacity': 12,
                   'Mass': 13,
                   'Nodule': 14,
                   'Pleural_Thickening': 15,
                   'Pneumonia': 16,
                   'Pneumothorax': 17}

file_features_names = open(output_features_names_pth,'w')
for feature_name in feature_names:
    file_features_names.write(feature_name)
    file_features_names.write('\n')
file_features_names.close()

# Extract from mid-layer

## Extract features 

In [ ]:
import extract_from_mid_layer

In [ ]:
imgs_path = "./output/not_went_icu_sel_proc_aug/"
features_midlayer_output_path = './output/went_icu_sel_proc_aug_mid.csv'

extract_from_mid_layer.extract_features(imgs_path,features_midlayer_output_path)

## Extract heatmaps

In [ ]:
# imgs_path_dic = {'pth1': "../../output/covid_not_icu_images/",
#                  'pth2': "../../output/covid_icu_images/"
#                 }
# saliency_path_dic = {'pth1': "../../output/heatmaps1/",
#                      'pth2': "../../output/heatmaps2/"
#                     }

# which_feature = 4
# gaussian_filter_size = 4

# extract_from_last_layer.extract_heatmap(imgs_path_dic, saliency_path_dic, which_feature, gaussian_filter_size)

## Extract features from heatmaps

In [ ]:
import heatmap_features

In [ ]:
features_name = {'Atelectasis': 0,
                   'Cardiomegaly': 1,
                   'Consolidation': 2,
                   'Edema': 3,
                   'Effusion': 4,
                   'Emphysema': 5,
                   'Enlarged Cardiomediastinum': 6,
                   'Fibrosis': 7,
                   'Fracture': 8,
                   'Hernia': 9,
                   'Infiltration': 10,
                   'Lung Lesion': 11,
                   'Lung Opacity': 12,
                   'Mass': 13,
                   'Nodule': 14,
                   'Pleural_Thickening': 15,
                   'Pneumonia': 16,
                   'Pneumothorax': 17}

sub_features_name = {'whole_image_entropy': 0,
                   'whole_image_energy': 1,
                   'long1_entropy': 2,
                   'long1_energy': 3,
                   'long2_entropy': 4,
                   'long2_energy': 5,
                   'trans1_entropy': 6,
                   'trans1_energy': 7,
                   'trans2_entropy': 8,
                   'trans2_energy': 9,
                   'quad1_entropy': 10,
                   'quad1_energy': 11,
                   'quad2_entropy': 12,
                   'quad2_energy': 13,
                   'quad3_entropy': 14,
                   'quad3_energy': 15,
                   'quad4_entropy': 16,
                   'quad4_energy': 17,
                   'quad5_entropy': 18,
                   'quad5_energy': 19,
                   'quad6_entropy': 20,
                   'quad6_energy': 21}

In [ ]:
# #Print feature names to get later after fitting tree

features_names = []
for i in range(len(features_name)):
    for j in range(len(sub_features_name)):
        features_names.append(list(features_name.keys())[i] + '_' + list(sub_features_name.keys())[j])


output_features_names_pth = './output/features_names_heatmap.txt'
file_features_names = open(output_features_names_pth,'w+')

for name in features_names:
    file_features_names.write(name)
    file_features_names.write('\n')
file_features_names.close()

In [ ]:
imgs_path = "./output/not_went_icu_sel_proc_aug/"
heatmap_features_output_path = './output/not_went_icu_sel_proc_aug_heatmap.csv'

sigma_ = 16
truncate_ = 3.5

heatmap_features.extract_heatmap_features(features_name, sub_features_name, imgs_path, heatmap_features_output_path, sigma_, truncate_)